In [1]:
import pandas as pd
import numpy as np

In [2]:
cancer = pd.read_csv('../data/raw/string_cancer.csv', sep=';')
pre = pd.read_csv('../data/raw/string_preeclampsia.csv', sep=';')

rename_dict = {'#node1': 'node1', 'node1_string_id':'node1_id', 'node2_string_id':'node2_id',}
cancer = cancer.rename(columns=rename_dict)
pre = pre.rename(columns=rename_dict)

In [3]:
# load a xls file as pandas and get a specific tab
xls = pd.ExcelFile('../data/raw/proteomica.xlsx')
xls_cancer = pd.read_excel(xls, 'Cancer-Rat').drop(['Nome da proteína (caso tenha que usar o nome do gene no string)', 'Unnamed: 5', 'Unnamed: 6'], axis=1)
xls_pre = pd.read_excel(xls, 'Preeclampsia-Rat').drop(['Gene name', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], axis=1).rename(columns={'Gene name.1': 'Gene name'})
xls_pre['Gene name'] = xls_pre['Gene name'].str.strip()
xls_cancer

,Query protein,Gene name,FC,Regulation
0,Q6AYZ1,Tuba1c,-0.440370,Down
1,Q9EPH1,A1bg,-0.778146,Down
2,P06238,A2m,-0.711127,Down
3,G3V9J6,Abcb1b,-0.557322,Down
4,P68136,Acta1,-0.284675,Down
...,...,...,...,...
172,D4A3P1,Ubqln4,-0.528368,Down
173,D4ABI6,Uchl4,-0.634966,Down
174,Q9Z270,Vapa,-0.838787,Down
175,Q3MIE4,Vat1,-0.511388,Down


In [4]:
xls_cancer['Gene name'].nunique(), xls_pre['Gene name'].nunique()

(176, 316)

In [5]:
xls_pre['Gene name'].value_counts().sort_values(ascending=False)

Gene name
Gbe1      2
Gpx3      2
Tes       1
Tuba4a    1
Uso1      1
         ..
Casp6     1
Gar1      1
Pappa2    1
Abcb4     1
Kng1      1
Name: count, Length: 316, dtype: int64

In [6]:
len(set(cancer.node1.unique().tolist() + cancer.node2.unique().tolist()))

104

In [7]:
len(set(pre.node1.unique().tolist() + pre.node2.unique().tolist()))

153

In [8]:
# set(xls_cancer['Gene name'].unique().tolist()) - set(cancer.node1.unique().tolist() + cancer.node2.unique().tolist())

In [9]:
# set(xls_pre['Gene name'].unique().tolist()) - set(pre.node1.unique().tolist() + pre.node2.unique().tolist())

In [10]:
xls_cancer.columns, xls_pre.columns

(Index(['Query protein', 'Gene name', 'FC', 'Regulation'], dtype='object'),
 Index(['Protein (UNIPROT)', 'protein_firstname', 'P value', 'Regulated',
        'Fold change', 'Obs.', 'Gene name'],
       dtype='object'))

In [11]:
xls_cancer = xls_cancer.rename(columns={'Gene name': 'gene', 'FC': 'fc'})
xls_cancer = xls_cancer.drop(['Regulation', 'Query protein'], axis=1)
# xls_cancer['fc'] = xls_cancer['fc'].str.replace(',', '.').astype(float)
xls_cancer['regulated'] = np.where(xls_cancer['fc'] > 0, 'up', 'down')
xls_cancer['cancer'] = 1
xls_cancer.columns

Index(['gene', 'fc', 'regulated', 'cancer'], dtype='object')

In [12]:
xls_pre = xls_pre.drop(['Protein (UNIPROT)', 'protein_firstname','P value','Obs.'], axis=1)
xls_pre = xls_pre.rename(columns={'Gene name': 'gene', 'Fold change': 'fc', 'Regulated': 'regulated'})
xls_pre['preeclampsia'] = 1
xls_pre['regulated'] = xls_pre['regulated'].str.lower()
# xls_pre['fc'] = xls_pre['fc'].str.replace(',', '.').astype(float)
xls_pre.columns

Index(['regulated', 'fc', 'gene', 'preeclampsia'], dtype='object')

In [13]:
xls_cancer.gene.nunique(), xls_pre.gene.nunique()

(176, 316)

Number of unique proteins in node1 and node2

In [14]:
cancer.node1.nunique(), cancer.node2.nunique()

(72, 70)

Checking if there are genes in string that wasnt in the disease file

In [15]:
string_genes = set(cancer.node1.unique().tolist() + cancer.node2.unique().tolist())
xls_genes = set(xls_cancer.gene.unique().tolist())
string_genes - xls_genes

set()

In [16]:
string_genes = set(pre.node1.unique().tolist() + pre.node2.unique().tolist())
xls_genes = set(xls_pre.gene.unique().tolist())
string_genes - xls_genes

set()

Genes that are present only in node1 or node2

In [17]:
diff = set(cancer.node1.unique()).symmetric_difference(set(cancer.node2.unique()))
len(diff)

66

Number of unique proteins(gene?)

In [18]:
len(set(cancer.node1.unique().tolist() + cancer.node2.unique().tolist()))

104

In [19]:
len(set(pre.node1.unique().tolist() + pre.node2.unique().tolist()))

153

In [20]:
f_df = xls_cancer.merge(xls_pre, on='gene', suffixes=('_cancer', '_pre'), how='outer')
f_df.cancer = f_df.cancer.fillna(0)
f_df.preeclampsia = f_df.preeclampsia.fillna(0)
f_df.to_csv('../data/processed/genes.csv', index=False)
f_df

,gene,fc_cancer,regulated_cancer,cancer,regulated_pre,fc_pre,preeclampsia
0,Tuba1c,-0.440370,down,1.0,NaN,NaN,0.0
1,Tuba1c,-0.355256,down,1.0,NaN,NaN,0.0
2,A1bg,-0.778146,down,1.0,NaN,NaN,0.0
3,A2m,-0.711127,down,1.0,NaN,NaN,0.0
4,Abcb1b,-0.557322,down,1.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...
476,Rpl34,NaN,NaN,0.0,up,0.400813,1.0
477,Casp6,NaN,NaN,0.0,up,0.559243,1.0
478,Gar1,NaN,NaN,0.0,up,0.580970,1.0
479,Pappa2,NaN,NaN,0.0,up,0.811140,1.0


In [21]:
df_no_nans = f_df.dropna()
print(len(df_no_nans))
df_no_nans.sort_values('regulated_cancer', ascending=False)
df_no_nans.to_csv('common_genes.csv', index=False)

22


In [23]:
cancer.to_csv('../data/processed/string_cancer.csv', index=False)
pre.to_csv('../data/processed/string_pre.csv', index=False)

### Adding a column that determines if the gene is present in both graphs
Requires the files already processed in Neo4J

In [24]:
graph = pd.read_csv('../data/processed/nodes.csv')
graph['present_in'] = np.where(graph['n.cancer'] == 0, 'pre', np.where(graph['n.preeclampsia'] == 0, 'cancer', 'both'))
graph.to_csv('../data/processed/nodes.csv', index=False)

In [25]:
graph

,n.name,n.cancer,n.preeclampsia,n.foldc_cancer,n.foldc_pre,n.regulated_cancer,n.regulated_pre,present_in
0,Tuba1c,1,0,-0.355256,NaN,down,NaN,cancer
1,A1bg,1,0,-0.778146,NaN,down,NaN,cancer
2,A2m,1,0,-0.711127,NaN,down,NaN,cancer
3,Abcb1b,1,0,-0.557322,NaN,down,NaN,cancer
4,Acta1,1,0,-0.284675,NaN,down,NaN,cancer
...,...,...,...,...,...,...,...,...
465,Rpl34,0,1,NaN,0.400813,NaN,up,pre
466,Casp6,0,1,NaN,0.559243,NaN,up,pre
467,Gar1,0,1,NaN,0.580970,NaN,up,pre
468,Pappa2,0,1,NaN,0.811140,NaN,up,pre


In [26]:
graph.present_in.value_counts()

present_in
pre       294
cancer    154
both       22
Name: count, dtype: int64